### MRMR-More

More experiments with the MRMR codebase.

In [ ]:
import sys
sys.path.append("../..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

In [ ]:
# Run an experiment with a different budget
Config().copy_experiment("mrmr")
Config().set_experiment_path(pathlib.Path(Config()[""]))
Config().set_experiment_data()